## 🎨 Visualization & Advanced Features

Beyond basic data loading, Maya4 provides powerful visualization tools and advanced sampling strategies.

## 📚 Quick Reference Card

Copy-paste ready configurations for common use cases.

In [ ]:
"""
==============================================================================
📚 QUICK REFERENCE: COPY-PASTE READY CONFIGURATIONS
==============================================================================

Common configurations for different research scenarios. Simply copy and modify!
"""

from maya4 import get_sar_dataloader, SARTransform, SampleFilter
import functools
from maya4 import minmax_normalize, RC_MIN, RC_MAX, GT_MIN, GT_MAX
import os
import pandas as pd

DATA_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))

# ============================================================================
# 🎯 CONFIG 1: FAST PROTOTYPING (Single Product, Quick Testing)
# ============================================================================
print('='*80)
print('🎯 Configuration 1: Fast Prototyping')
print('='*80 + '\n')

loader_prototype = get_sar_dataloader(
    data_dir=DATA_DIR,
    level_from="rcmc",              # Input: Range Cell Migration Corrected
    level_to="az",                  # Output: Azimuth focused (focused image)
    batch_size=4,                   # Small batch for quick iteration
    num_workers=0,                  # Single process for easy debugging
    patch_size=(1000, 1000),        # 1km × 1km patches
    stride=(1000, 1000),            # Non-overlapping patches
    buffer=(500, 500),              # 500px boundary buffer
    patch_order="chunk",            # I/O efficient ordering
    complex_valued=True,            # Native SAR complex representation
    online=True,                    # Auto-download from HuggingFace
    max_products=1,                 # Just 1 product for testing
    samples_per_prod=50,            # 50 samples per product
    cache_size=1,                   # Minimal cache
    verbose=True,                   # Show debug information
    use_balanced_sampling=False     # No balancing for single product
)
print('✓ Fast prototyping loader ready\n')

# ============================================================================
# 🚀 CONFIG 2: FULL TRAINING (Production-Ready, Multi-Product)
# ============================================================================
print('='*80)
print('🚀 Configuration 2: Full Training Setup')
print('='*80 + '\n')

# Define normalization transforms
full_training_transforms = SARTransform(
    transform_rcmc=functools.partial(minmax_normalize, array_min=RC_MIN, array_max=RC_MAX),
    transform_az=functools.partial(minmax_normalize, array_min=GT_MIN, array_max=GT_MAX)
)

# Define product filters
training_filters = SampleFilter(
    years=[2023, 2024],             # Multi-year dataset
    polarizations=["hh", "hv"],     # Multiple polarizations
    stripmap_modes=[1, 2, 3],       # Various swath modes
    parts=None                       # All geographic regions
)

loader_training = get_sar_dataloader(
    data_dir=DATA_DIR,
    filters=training_filters,
    transform=full_training_transforms,
    level_from="rcmc",
    level_to="az",
    batch_size=32,                  # Larger batch for GPU efficiency
    num_workers=4,                  # Parallel loading
    patch_size=(1000, 1000),
    stride=(500, 500),              # 50% overlap for more samples
    buffer=(1000, 1000),
    patch_order="chunk",
    shuffle_files=True,             # Randomize product order
    complex_valued=True,
    online=True,
    max_products=None,              # Use all available products
    samples_per_prod=1000,          # Dense sampling
    cache_size=50,                  # Cache multiple products
    verbose=False,                  # Quiet mode for training
    use_balanced_sampling=True      # Geographic balance (requires sklearn)
)
print('✓ Full training loader ready\n')

# ============================================================================
# 🔬 CONFIG 3: VERTICAL SLICE ANALYSIS (Range Direction)
# ============================================================================
print('='*80)
print('🔬 Configuration 3: Vertical Slice Analysis')
print('='*80 + '\n')

loader_vertical = get_sar_dataloader(
    data_dir=DATA_DIR,
    level_from="rcmc",
    level_to="az",
    batch_size=16,
    num_workers=0,
    patch_mode="rectangular",
    patch_size=(1, -1),             # Full width, 1-pixel height (horizontal slices)
    buffer=(1000, 0),               # Buffer only top/bottom
    stride=(1, 1),                  # Sample every row
    patch_order="row",              # Row-by-row sampling
    complex_valued=True,
    online=True,
    max_products=1,
    samples_per_prod=100,
    cache_size=1,
    verbose=False
)
print('✓ Vertical slice loader ready')
print('  Output shape: (batch, 2, 1, full_width) or (batch, 1, full_width) if complex\n')

# ============================================================================
# 📊 CONFIG 4: HORIZONTAL SLICE ANALYSIS (Azimuth Direction)
# ============================================================================
print('='*80)
print('📊 Configuration 4: Horizontal Slice Analysis')
print('='*80 + '\n')

loader_horizontal = get_sar_dataloader(
    data_dir=DATA_DIR,
    level_from="rcmc",
    level_to="az",
    batch_size=16,
    num_workers=0,
    patch_mode="rectangular",
    patch_size=(-1, 1),             # Full height, 1-pixel width (vertical slices)
    buffer=(0, 1000),               # Buffer only left/right
    stride=(1, 1),                  # Sample every column
    patch_order="col",              # Column-by-column sampling
    complex_valued=True,
    online=True,
    max_products=1,
    samples_per_prod=100,
    cache_size=1,
    verbose=False
)
print('✓ Horizontal slice loader ready')
print('  Output shape: (batch, 2, full_height, 1) or (batch, full_height, 1) if complex\n')

# ============================================================================
# 🎓 CONFIG 5: SEQUENCE LEARNING (With Positional Encoding)
# ============================================================================
print('='*80)
print('🎓 Configuration 5: Sequence Learning (Transformer-Ready)')
print('='*80 + '\n')

loader_sequence = get_sar_dataloader(
    data_dir=DATA_DIR,
    level_from="rcmc",
    level_to="az",
    batch_size=8,
    num_workers=0,
    patch_size=(100, 1),            # Thin vertical patches
    buffer=(1000, 1000),
    stride=(100, 1),                # Non-overlapping
    patch_order="col",              # Column-wise for sequences
    complex_valued=False,           # Real/imag as separate channels
    positional_encoding=True,       # Add spatial coordinates
    concatenate_patches=True,       # Stack patches into sequences
    concat_axis=0,                  # Concatenate vertically
    max_base_sample_size=(5000, 1000),  # Limit sequence length
    online=True,
    max_products=2,
    samples_per_prod=100,
    cache_size=2,
    verbose=False
)
print('✓ Sequence learning loader ready')
print('  Output shape: (batch, channels, sequence_length, feature_dim)')
print('  Positional encoding adds 2 extra channels with (y, x) coordinates\n')

# ============================================================================
# 🧩 CONFIG 6: BLOCK PATTERN SAMPLING (Stratified Coverage)
# ============================================================================
print('='*80)
print('🧩 Configuration 6: Block Pattern Sampling')
print('='*80 + '\n')

loader_blocks = get_sar_dataloader(
    data_dir=DATA_DIR,
    level_from="rcmc",
    level_to="az",
    batch_size=16,
    num_workers=0,
    patch_size=(100, 100),
    buffer=(1000, 1000),
    stride=(100, 100),
    block_pattern=(32, 32),         # Divide into 32×32 grid of blocks
    patch_order="row",              # Within each block
    complex_valued=True,
    online=True,
    max_products=1,
    samples_per_prod=200,
    cache_size=1,
    verbose=False
)
print('✓ Block pattern loader ready')
print('  Ensures stratified sampling across entire SAR product')
print('  Good for capturing diverse spatial characteristics\n')

# ============================================================================
# 🔄 CONFIG 7: END-TO-END LEARNING (Raw → Focused)
# ============================================================================
print('='*80)
print('🔄 Configuration 7: End-to-End SAR Processing')
print('='*80 + '\n')

e2e_transforms = SARTransform(
    transform_raw=functools.partial(minmax_normalize, array_min=RC_MIN, array_max=RC_MAX),
    transform_az=functools.partial(minmax_normalize, array_min=GT_MIN, array_max=GT_MAX)
)

loader_e2e = get_sar_dataloader(
    data_dir=DATA_DIR,
    level_from="raw",               # Start from Level 0 raw data
    level_to="az",                  # End with Level 1 focused image
    transform=e2e_transforms,
    batch_size=8,
    num_workers=0,
    patch_size=(2000, 2000),        # Larger patches for full processing
    buffer=(1000, 1000),
    stride=(2000, 2000),
    patch_order="chunk",
    complex_valued=True,
    online=True,
    max_products=2,
    samples_per_prod=50,
    cache_size=2,
    verbose=False
)
print('✓ End-to-end loader ready')
print('  Most challenging task: complete focusing pipeline')
print('  Requires learning all intermediate processing steps\n')
